# Exploring building reference datasets with kerchunk

The idea: build a test intake catalogue of kerchunk virtual dataset(s) from the COSIMA output(s) and see how it performs

In [1]:
import os

import dask

import glob

import fsspec

import ujson

import xarray as xr

from distributed import Client

from kerchunk.hdf import SingleHdf5ToZarr
from kerchunk.combine import MultiZarrToZarr

# Build a reference dataset for COSIMA `access-om2-025` `025deg_jra55_iaf_omip2_cycle1` `ocean_month` data (~2.2TB, 61 netcdf files)

In [2]:
exp_root = "/g/data/ik11/outputs/access-om2-025/025deg_jra55_iaf_omip2_cycle1"

fs = fsspec.filesystem('file')
flist = fs.glob(f"{exp_root}/output*/ocean/ocean_month.nc")

## Write single file jsons in parallel

In [3]:
%%time

@dask.delayed
def gen_json(file):
    with fs.open(file) as infile:
        h5chunks = SingleHdf5ToZarr(infile, file)
        outf = f"{'.'.join(file.split('/')[5:])}.json"
        with open(outf, 'wb') as f:
            f.write(ujson.dumps(h5chunks.translate()).encode());
            
# This would take well over an hour without dask
with Client(n_workers=8):
    dask.compute(*[gen_json(file) for file in flist])

CPU times: user 1min 19s, sys: 13.7 s, total: 1min 32s
Wall time: 13min 51s


## Combine into one multi-file json

In [4]:
%%time

json_list = fs.glob("./access-om2-025.025deg_jra55_iaf_omip2_cycle1.*.ocean.ocean_month.nc.json")

mzz = MultiZarrToZarr(
    json_list,
    concat_dims=['time'],
    identical_dims=[
        "xt_ocean", 
        "yt_ocean", 
        "st_ocean", 
        "xu_ocean", 
        "yu_ocean", 
        "sw_ocean", 
        "grid_xt_ocean", 
        "grid_yt_ocean", 
        "grid_xu_ocean", 
        "grid_yu_ocean", 
        "potrho", 
        "neutral",
        "nv"
     ],
)

d = mzz.translate("access-om2-025.025deg_jra55_iaf_omip2_cycle1.ocean_month.json") # A 1.1 GB json!

for json in json_list:
    os.remove(json)

CPU times: user 1min 57s, sys: 50.4 s, total: 2min 47s
Wall time: 2min 43s


## Load the reference dataset and do something

In [3]:
client = Client(processes=False)
print(f"Dask dashboard at: {client.dashboard_link}")

Dask dashboard at: http://10.6.79.58:8787/status


In [4]:
%%time

m = fsspec.get_mapper(
    'reference://', 
    fo="access-om2-025.025deg_jra55_iaf_omip2_cycle1.ocean_month.json", 
    remote_protocol="file"
)
ds = xr.open_dataset(
    m,
    engine='zarr', 
    backend_kwargs={"consolidated": False},
    chunks={"time": -1},
    decode_times=False
)

CPU times: user 21.5 s, sys: 2.77 s, total: 24.3 s
Wall time: 24.2 s


In [5]:
%%time

# This compute comprises 44 dask tasks and uses 4GB of unmanaged memory
global_mean = ds["sst"].mean(["xt_ocean", "yt_ocean"]).compute()

CPU times: user 59.1 s, sys: 52.8 s, total: 1min 51s
Wall time: 47.7 s


## Compare to `open_mfdataset`

In [4]:
%%time

ds = xr.open_mfdataset(
    flist,
    chunks={"time": -1},
    concat_dim="time",
    parallel=True,
    combine="nested",
    data_vars="minimal", 
    coords="minimal", 
    compat="override"
)

CPU times: user 21.3 s, sys: 9.25 s, total: 30.5 s
Wall time: 19.6 s


In [5]:
%%time

# This compute comprises 305 dask tasks and uses 2GB of unmanaged memory
global_mean = ds["sst"].mean(["xt_ocean", "yt_ocean"]).compute()

CPU times: user 22.3 s, sys: 3.33 s, total: 25.7 s
Wall time: 15.1 s


## Thoughts

- The main impediment to this sort of approach at the moment is that kerchunk references are stored as inefficient json files which can be very large for datasets comprising many chunks. Currently this entire file is loaded with `open_dataset` (and loaded onto each worker when using a distributed cluster) which can require lots of memory and overhead. This outweighs the benefits kerchunk provides in this application (consolidation + zarr simplicity/performance).

- This said, there are a number of things in the kerchunk pipeline that should address the above issue: 1) new data structures for the reference set(s) that facilitate lazy loading (see https://github.com/fsspec/kerchunk/issues/240 and https://github.com/fsspec/kerchunk/issues/134), 2) the ability the split/combine chunks within a reference file but this would have limitations (e.g. couldn't work for particular types of compression, see https://github.com/fsspec/kerchunk/issues/124 and https://github.com/fsspec/kerchunk/issues/134).

- The approach of generating reference datasets for new experiments should be relatively straight-forward if we can have some confidence that all input files will be readily "concatenatable" in the sense that they have the same chunking patterns, coordinates on fixed dims, fixed auxiliary coordinates, variables (where approapriate) etc. The latest functionality for combining references in kerchunk is pretty versatile, see https://fsspec.github.io/kerchunk/tutorial.html#using-coo-map.

- Where the raw data are unchunked (e.g. netcdf3), this approach can significantly impact performance since fsspec will make the full range request even for subsets (https://github.com/fsspec/kerchunk/issues/124).

In [6]:
client.close()